# 🧬 Solution_RD: Genetic Algorithm Individual for Resource Distribution

This notebook documents the `Solution_RD` class, a domain-specific implementation of the `Individual` abstract base class tailored for solving a resource distribution problem involving participants, houses, and courses. The objective balances logistical and social criteria using a flexible and pluggable design.

## 🧱 Class Overview

- `Solution_RD`: A concrete subclass of `Individual` that represents a candidate solution for assigning participants to houses and courses.
  - Representation encodes both **house allocation** and **course enrollment**.
  - Fitness balances two components:
    - **Social fitness**: Encourages participant mixing across courses.
    - **Logistic fitness**: Penalizes long distances between participant houses (based on a distance matrix).
  - Customizable fitness trade-off via `alpha` and `beta`.

## ⚙️ Parameters

- `data_matrix`: A 2D numpy array of distances between participants.
- `account_for_previous_fitness`: If `True`, tracks fitness changes over time.
- `probability_social_mutation`: Probability of performing a social mutation.
- `alpha`, `beta`: Weights controlling the importance of social vs. logistic fitness.

## ✅ Features Demonstrated

1. **Random Initialization**
   - Genome includes both house and course assignments.
   - Ensures constraints like house capacity and number of houses are respected.

2. **Fitness Evaluation**
   - Calculated as a weighted sum of:
     - Social interaction score (maximize diversity in house-course assignments).
     - Logistic penalty (minimize travel distances).
   - Supports lazy evaluation and caching.

3. **Mutation**
   - Mutation type (social or logistic) chosen probabilistically.
   - Mutates in-place.

4. **Crossover**
   - Combines genome segments from two parents (to be implemented or overridden if necessary).

5. **Parameterization**
   - Adjust evolutionary pressure via `alpha`, `beta`, and mutation probabilities.

6. **Fitness History**
   - Tracks `previous_fitness` and `next_fitness` for advanced evolutionary strategies.

## 🧠 Design Highlights

- Extends `Individual` with problem-specific logic, maintaining modularity.
- Balances domain constraints with genetic flexibility.
- Supports experimentation with different fitness weights and mutation strategies.

This class is suitable for constrained optimization tasks in social or logistical planning domains and can be integrated seamlessly into the larger genetic algorithm framework.


# Data Ingestion

In [ ]:
import os
import pandas as pd
os.chdir(os.pardir) # comment if you are running this more than once
data_matrix_df = pd.read_csv('data/distance_matrix_official.csv', index_col=0)
data_matrix_np = data_matrix_df.to_numpy()
data_matrix_np.shape

# Testing Features

## Create two individuals


In [ ]:
from Genetic_algorithm.fitness import ResourceFitness
from Genetic_algorithm.genome import Genome
from Genetic_algorithm.solution_rd import SolutionRD
from Genetic_algorithm.mutations import logistic_mutation, social_mutation, logistic_mutation_2
from Genetic_algorithm.crossovers import social_crossover, logistic_crossover_2, full_crossover

my_fitness = ResourceFitness(data_matrix_np)
my_genome = Genome()

first_solution = SolutionRD(my_fitness, Genome, [logistic_mutation, social_mutation], [logistic_crossover_2, social_crossover])
secomnd_solution = SolutionRD(my_fitness, Genome, [logistic_mutation, social_mutation], [logistic_crossover_2, social_crossover])

## Crossover


In [ ]:
for _ in range(1000):
    child1, child2 = first_solution @ secomnd_solution
    child1.mutation()
print("Fitness (child):", float(child1))
print("Fitness (child):", float(child2))

## Mutation


In [ ]:
my_fitness.partecipation_matrix(genome=child1.genome)

In [ ]:
mutant = child1.copy_Individual()
mutant.mutation()
print("original Individual:", child1.genome)
print("Mutated Individual:", mutant.genome)
print("Fitness (original):", float(child1))
print("Fitness (mutated):", float(mutant))

## Iteration (generate mutated versions) (has to be resolved)

In [ ]:
print("\nMultiple Mutations:")
for i, m in enumerate(child1):
    print(f"Mutation {i+1}:", m, "Fitness:", m)

## Operator Overloading

In [ ]:
print("\nUsing ** Operator for Self-Crossover")
clones = child1 ** 3
for i, c1, in enumerate(clones):
    print(f"Clone {i+1}:", c1, "Fitness:", c1)
   

## Comparison


In [ ]:
print("\nIs Individual 1 better than Individual 2?", child1 > child2)

## Sorting and Deduplication via Set

In [ ]:
print("\n=== Sorting Individuals by Fitness ===")
individuals = [child for child in child1]

for i, ind in enumerate(individuals):
    print(f"Ind {i+1}: Genome = {ind.genome}, Fitness = {float(ind)}")

sorted_individuals = sorted(individuals)
print("\nSorted by fitness (ascending):")
for i, ind in enumerate(sorted_individuals):
    print(f"Rank {i+1}: Genome = {ind.genome}, Fitness = {float(ind)}")

## Storing Unique Individuals in a Set

In [ ]:
print("\n=== Deduplication with Set ===")
ind2 = child1.copy_Individual(delete_fitness=False)  # Exact copy

# Mutate a third individual to likely get a different genome
ind3 = child1.copy_Individual(delete_fitness=False)
ind3.mutation()

ind_set = {child1, ind2, ind3}

for i, ind in enumerate(ind_set):
    print(f"Unique {i+1}: Genome = {ind.genome}, Fitness = {float(ind)}")

print(f"\nTotal unique individuals: {len(ind_set)}")

# accessing the genome (removed)

In [ ]:
print("Before genome swap:", child1.genome.course_assignments)
child1.genome.swap_course_assignments(0,0, 1)
print("After genome swap:", child1.genome.course_assignments)

In [ ]:
try:    
    child1.genome.swap_course_assignments(0,0, 10000)
except IndexError as e:
    print("Error:", e)

# genetic algorithm

In [ ]:
import random
from typing import List

def tournament_selection(
    population: List[SolutionRD],
    maximization: bool = False,
    k: int = 3
) -> SolutionRD:
    """
    k-way tournament (with replacement), unified for minimization or maximization.

    Parameters
    ----------
    population : List[SolutionRD]
        Current population.
    maximization : bool, default=False
        If False, lower fitness is better (minimize). If True, higher fitness is better.
    k : int, default=3
        Tournament size (number of contestants).

    Returns
    -------
    SolutionRD
        The tournament winner.
    """
    # Randomly select k individuals from the population
    contestants = random.choices(population, k=k)
    
    # Determine the winner based on fitness
    if maximization:
        winner = max(contestants, key=lambda ind: float(ind))
    else:
        winner = min(contestants, key=lambda ind: float(ind))
    
    return winner


def rank_selection(
    population: List[SolutionRD],
    maximization: bool = False
) -> SolutionRD:
    """
    Rank-based roulette selection.

    1. Sort population by fitness.
    2. Assign weights N, N-1, …, 1 (best gets highest weight).
    3. Sample one individual proportionally to its weight.

    Parameters
    ----------
    population : List[Individual]
        Current population.
    maximization : bool, default False
        If False, lower fitness is better; if True, higher fitness is better.

    Returns
    -------
    Individual
        The selected individual.
    """
    # Sort population by fitness
    sorted_population = sorted(population, reverse=maximization)
    
    # Assign weights
    weights = list(range(len(sorted_population), 0, -1))
    
    # Calculate total weight
    total_weight = sum(weights)
    
    # Calculate selection probabilities
    probabilities = [weight / total_weight for weight in weights]
    
    # Select an individual based on the calculated probabilities
    return random.choices(sorted_population, weights=probabilities, k=1)[0]

    

first_solution = SolutionRD(my_fitness, Genome, [logistic_mutation, social_mutation], [logistic_crossover_2, social_crossover])
population = [first_solution.copy_Individual() for _ in range(10)]

In [ ]:
ranked_winner = rank_selection(population)
print("Ranked Winner:", ranked_winner.genome, "Fitness:", float(ranked_winner))

In [ ]:
tournament_winner = tournament_selection(population)
print("Ranked Winner:", ranked_winner.genome, "Fitness:", float(ranked_winner))

In [ ]:
 
def genetic_algorithm(
    gen_count: int,
    selection_algorithm: list[callable],
    mutation_algorithms: list[callable],
    crossover_algorithms: list[callable],
    fitness_instance: ResourceFitness,
    Genome_class: Genome,
    maximization: bool = False,
    xo_prob: float = 0.9,
    mut_prob: float = 0.2,
    social_mutation_prob: float = 0.2,
    social_crossover_prob: float = 0.2,
    population_size: int = 10,
    elitism: bool = True,
    verbose: bool = False,
):
    """
    Genetic Algorithm for solving optimization problems.
    This function implements a genetic algorithm that evolves a population of solutions
    over a specified number of generations. It uses selection, mutation, and crossover
    algorithms to create new solutions and improve the population's fitness.
    The algorithm can be configured with various parameters, including the number of
    generations, selection method, mutation and crossover probabilities, and whether
    to use elitism.

    Args:
        gen_count (int): number of generations to evolve.
        selection_algorithm (list[callable]): selection algorithm to use.
        mutation_algorithms (list[callable]): choose from Genetic_algotithm.mutations
        crossover_algorithms (list[callable]): choose from Genetic_algorithm.crossovers
        fitness_instance (ResourceFitness): instancee of the fitness class (it's important to instanciate it before as it holds the data matrix)
        Genome_class (Genome): class of the genome to be used (will be instanciated in the function)
        maximization (bool, optional): _description_. Defaults to False.
        xo_prob (float, optional): _description_. Defaults to 0.9.
        mut_prob (float, optional): _description_. Defaults to 0.2.
        social_mutation_prob (float, optional): _description_. Defaults to 0.2.
        social_crossover_prob (float, optional): _description_. Defaults to 0.2.
        elitism (bool, optional): _description_. Defaults to True.
        verbose (bool, optional): _description_. Defaults to False.
        
    """
    
    
    
    first_solution = SolutionRD(fitness_instance, Genome_class, mutation_algorithms, crossover_algorithms)
    first_solution.prob_social_mutation = social_mutation_prob
    first_solution.prob_social_crossover = social_crossover_prob
    
    GA_population = [first_solution.copy_Individual() for _ in range(population_size)]
    
    
    for generation in range(gen_count):
        if verbose:
            print(f"Generation {generation+1}/{gen_count}")
        
        # Selection
        selected_individuals = [selection_algorithm(GA_population, maximization) for _ in range(population_size)]
        
        # Crossover
        new_population = []
        for i in range(0, population_size):
            for j in range(i+1, population_size):
                if random.random() < xo_prob:
                    parent1 = selected_individuals[i]
                    parent2 = selected_individuals[j]
                    child1, child2 = parent1 @ parent2
                    new_population.append(child1)
                    new_population.append(child2)
                    
        
        # Mutation
        for individual in new_population:
            if random.random() < mut_prob:
                individual.mutation()
        
        # Elitism
        if elitism:
            best_individual = min(GA_population) if not maximization else max(GA_population)
            new_population[0] = best_individual
        
        GA_population = new_population
    
    # Return the best individual from the final population
    best_individual = min(GA_population) if not maximization else max(GA_population)
    return best_individual

# Example usage
best_solution = genetic_algorithm(
    gen_count=100,
    selection_algorithm=tournament_selection,
    mutation_algorithms=[logistic_mutation, social_mutation],
    crossover_algorithms=[logistic_crossover_2, social_crossover],
    fitness_instance=my_fitness,
    Genome_class=Genome,
    maximization=False,
    xo_prob=0.9,
    mut_prob=0.2,
    social_mutation_prob=0.2,
    social_crossover_prob=0.2,
    population_size=10,
    elitism=True,
    verbose=True
)
print("Best Solution Genome:", best_solution.genome)
print("Best Solution Fitness:", float(best_solution))
print("Average distance:", best_solution.fitness_instance._calculate_social_fitness(best_solution.genome))

